In [2]:
from datasets import load_dataset

# データセットを読み込む
dataset = load_dataset("llm-book/ner-wikipedia-dataset",trust_remote_code=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 4274
    })
    validation: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 534
    })
    test: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 535
    })
})


In [4]:
from pprint import pprint

pprint(list(dataset["train"])[:2])

[{'curid': '3638038',
  'entities': [{'name': 'さくら学院', 'span': [0, 5], 'type': 'その他の組織名'},
               {'name': 'Ciao Smiles', 'span': [6, 17], 'type': 'その他の組織名'}],
  'text': 'さくら学院、Ciao Smilesのメンバー。'},
 {'curid': '1729527',
  'entities': [{'name': 'レクレアティーボ・ウェルバ', 'span': [17, 30], 'type': 'その他の組織名'},
               {'name': 'プリメーラ・ディビシオン', 'span': [32, 44], 'type': 'その他の組織名'}],
  'text': '2008年10月5日、アウェーでのレクレアティーボ・ウェルバ戦でプリメーラ・ディビシオンでの初得点を決めた。'}]


In [5]:
from collections import Counter
import pandas as pd
from datasets import Dataset

def count_label_occurrences(dataset: Dataset) -> dict[str, int]:
    """固有表現タイプの出現回数をカウント"""
    # 各事例から固有表現タイプを抽出したlistを作成する
    entities = [
        e["type"] for data in dataset for e in data["entities"]
    ]

    # ラベルの出現回数が多い順に並び替える
    label_counts = dict(Counter(entities).most_common())
    return label_counts

label_counts_dict = {}
for split in dataset: # 各分割セットを処理する
    label_counts_dict[split] = count_label_occurrences(dataset[split])
# DataFrame形式で表示する
df = pd.DataFrame(label_counts_dict)
df.loc["合計"] = df.sum()
display(df)

,train,validation,test
人名,2394,299,287
法人名,2006,231,248
地名,1769,184,204
政治的組織名,953,121,106
製品名,934,123,158
施設名,868,103,137
その他の組織名,852,99,100
イベント名,831,85,93
合計,10607,1245,1333


In [6]:
from unicodedata import normalize

# テキストに対してUnicode正規化を行う
text = "ＡＢＣABCａｂｃabcｱｲｳアイウ①②③123"
normalized_text = normalize("NFKC", text)
print(f"正規化前: {text}")
print(f"正規化後: {normalized_text}")

正規化前: ＡＢＣABCａｂｃabcｱｲｳアイウ①②③123
正規化後: ABCABCabcabcアイウアイウ123123


In [7]:
text = "㈱、3㌕、10℃"
normalized_text = normalize("NFKC", text)
print(f"正規化前: {text}")
print(f"正規化後: {normalized_text}")

正規化前: ㈱、3㌕、10℃
正規化後: (株)、3キログラム、10°C


In [8]:
from transformers import AutoTokenizer

# トークナイザを読み込む
model_name = "cl-tohoku/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# トークナイゼーションを行う
subwords = "/".join(tokenizer.tokenize(dataset["train"][0]["text"]))
characters = "/".join(dataset["train"][0]["text"])
print(f"サブワード単位: {subwords}")
print(f"文字単位: {characters}")


サブワード単位: さくら/学院/、/C/##ia/##o/Sm/##ile/##s/の/メンバー/。
文字単位: さ/く/ら/学/院/、/C/i/a/o/ /S/m/i/l/e/s/の/メ/ン/バ/ー/。


/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
from spacy_alignments import get_alignments
text = "さくら学院"
characters = list(text)
tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))

char_to_token_indices, token_to_char_indices = get_alignments(characters, tokens)
print(f"文字のlist: {characters}")
print(f"トークンのlist: {tokens}")
print(f"文字に対するトークンの位置: {char_to_token_indices}")
print(f"トークンに対する文字の位置: {token_to_char_indices}")

文字のlist: ['さ', 'く', 'ら', '学', '院']
トークンのlist: ['[CLS]', 'さくら', '学院', '[SEP]']
文字に対するトークンの位置: [[1], [1], [1], [2], [2]]
トークンに対する文字の位置: [[], [0, 1, 2], [3, 4], []]


In [13]:
text = "大谷翔平は岩手県水沢市出身"
entities = [
    {"name": "大谷翔平", "span": [0, 4], "type": "人名"},
    {"name": "岩手県水沢市", "span": [5, 11], "type": "地名"},
]

In [15]:
from transformers import PreTrainedTokenizer

def output_tokens_and_labels(
    text: str,
    entities: list[dict[str, list[int] | str]],
    tokenizer: PreTrainedTokenizer,
) -> tuple[list[str], list[str]]:
    """トークンのlistとラベルのlistを出力"""
    # 文字のlistとトークンのlistのアライメントをとる
    characters = list(text)
    tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))
    char_to_token_indices, _ = get_alignments(characters, tokens)

    # "O"のラベルで初期化したラベルのlistを作成する
    labels = ["O"] * len(tokens)
    for entity in entities: # 各固有表現で処理する
        entity_span, entity_type = entity["span"], entity["type"]
        start = char_to_token_indices[entity_span[0]][0]
        end = char_to_token_indices[entity_span[1] - 1][0]
        # 固有表現の開始トークンの位置に"B-"のラベルを設定する
        labels[start] = f"B-{entity_type}"
        # 固有表現の開始トークン以外の位置に"I-"のラベルを設定する
        for idx in range(start + 1, end + 1):
            labels[idx] = f"I-{entity_type}"
    # 特殊トークンの位置にはラベルを設定しない
    labels[0] = "-"
    labels[-1] = "-"
    return tokens, labels

# トークンとラベルのlistを出力する
tokens, labels = output_tokens_and_labels(text, entities, tokenizer)
# DataFrameの形式で表示する
df = pd.DataFrame({"トークン列": tokens, "ラベル列": labels})
df.index.name = "位置"
display(df.T)

位置,0,1,2,3,4,5,6,7,8,9,10
トークン列,[CLS],大谷,翔,##平,は,岩手,県,水沢,市,出身,[SEP]
ラベル列,-,B-人名,I-人名,I-人名,O,B-地名,I-地名,I-地名,I-地名,O,-


In [17]:
from typing import Any
from seqeval.metrics import classification_report

def create_character_labels(
    text: str, entities: list[dict[str, list[int] | str]]
) -> list[str]:
    """文字ベースでラベルのlistを作成"""
    # "O"のラベルで初期化したラベルのlistを作成する
    labels = ["O"] * len(text)
    for entity in entities: # 各固有表現を処理する
        entity_span, entity_type = entity["span"], entity["type"]
        # 固有表現の開始文字の位置に"B-"のラベルを設定する
        labels[entity_span[0]] = f"B-{entity_type}"
        # 固有表現の開始文字以外の位置に"I-"のラベルを設定する
        for i in range(entity_span[0] + 1, entity_span[1]):
            labels[i] = f"I-{entity_type}"
    return labels

def convert_results_to_labels(
    results: list[dict[str, Any]]
) -> tuple[list[list[str]], list[list[str]]]:
    """正解データと予測データのラベルのlistを作成"""
    true_labels, pred_labels = [], []
    for result in results: # 各事例を処理する
        # 文字ベースでラベルのリストを作成してlistに加える
        true_labels.append(
            create_character_labels(result["text"], result["entities"])
        )
        pred_labels.append(
            create_character_labels(result["text"], result["pred_entities"])
        )
    return true_labels, pred_labels

In [19]:
from seqeval.metrics import f1_score, precision_score, recall_score

def compute_scores(
    true_labels: list[list[str]], pred_labels: list[list[str]], average: str
) -> dict[str, float]:
    """適合率、再現率、F値を算出"""
    scores = {
        "precision": precision_score(true_labels, pred_labels, average=average),
        "recall": recall_score(true_labels, pred_labels, average=average),
        "f1-score": f1_score(true_labels, pred_labels, average=average),
    }
    return scores


In [21]:
import torch

def create_label2id(
    entities_list: list[list[dict[str, str | int]]]
) -> dict[str, int]:
    """ラベルとIDを紐付けるdictを作成"""
    # "O"のIDには0を割り当てる
    label2id = {"O": 0}
    # 固有表現タイプのsetを獲得して並び替える
    entity_types = set(
        [e["type"] for entities in entities_list for e in entities]
    )
    entity_types = sorted(entity_types)
    for i, entity_type in enumerate(entity_types):
        # "B-"のIDには奇数番号を割り当てる
        label2id[f"B-{entity_type}"] = i * 2 + 1
        # "I-"のIDには偶数番号を割り当てる
        label2id[f"I-{entity_type}"] = i * 2 + 2
    return label2id

# ラベルとIDを紐付けるdictを作成する
label2id = create_label2id(dataset["train"]["entities"])
id2label = {v: k for k, v in label2id.items()}
pprint(id2label)

{0: 'O',
 1: 'B-その他の組織名',
 2: 'I-その他の組織名',
 3: 'B-イベント名',
 4: 'I-イベント名',
 5: 'B-人名',
 6: 'I-人名',
 7: 'B-地名',
 8: 'I-地名',
 9: 'B-政治的組織名',
 10: 'I-政治的組織名',
 11: 'B-施設名',
 12: 'I-施設名',
 13: 'B-法人名',
 14: 'I-法人名',
 15: 'B-製品名',
 16: 'I-製品名'}


In [25]:
from transformers.tokenization_utils_base import BatchEncoding

def preprocess_data(
    data: dict[str, Any],
    tokenizer: PreTrainedTokenizer,
    label2id: dict[int, str],
) -> BatchEncoding:
    """データの前処理"""
    # テキストのトークナイゼーションを行う
    inputs = tokenizer(
        data["text"],
        return_tensors="pt",
        return_special_tokens_mask=True,
    )
    inputs = {k: v.squeeze(0) for k, v in inputs.items()}

    # 文字のlistとトークンのlistのアライメントをとる
    characters = list(data["text"])
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
    char_to_token_indices, _ = get_alignments(characters, tokens)

    # "O"のIDのlistを作成する
    labels = torch.zeros_like(inputs["input_ids"])
    for entity in data["entities"]: # 各固有表現を処理する
        start_token_indices = char_to_token_indices[entity["span"][0]]
        end_token_indices = char_to_token_indices[
            entity["span"][1] - 1
        ]
        # 文字に対応するトークンが存在しなければスキップする
        if (
            len(start_token_indices) == 0
            or len(end_token_indices) == 0
        ):
            continue
        start, end = start_token_indices[0], end_token_indices[0]
        entity_type = entity["type"]
        # 固有表現の開始トークンの位置に"B-"のIDを設定する
        labels[start] = label2id[f"B-{entity_type}"]
        # 固有表現の開始トークン以外の位置に"I-"のIDを設定する
        if start != end:
            labels[start + 1 : end + 1] = label2id[f"I-{entity_type}"]
    # 特殊トークンの位置のIDは-100とする
    labels[torch.where(inputs["special_tokens_mask"])] = -100
    inputs["labels"] = labels
    return inputs

# 訓練セットに対して前処理を行う
train_dataset = dataset["train"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["train"].column_names,
)
# 検証セットに対して前処理を行う
validation_dataset = dataset["validation"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["validation"].column_names,
) 

Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, 'label2id': {'O': 0, 'B-その他の組織名': 1, 'I-その他の組織名': 2, 'B-イベント名': 3, 'I-イベント名': 4

Map:   0%|          | 0/4274 [00:00<?, ? examples/s]

Map:   0%|          | 0/534 [00:00<?, ? examples/s]

In [26]:
from transformers import (
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
)

# モデルを読み込む
model = AutoModelForTokenClassification.from_pretrained(
    model_name, label2id=label2id, id2label=id2label
)
# collate関数にDataCollatorForTokenClassificationを用いる
data_collator = DataCollatorForTokenClassification(tokenizer)

/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import set_seed

# 乱数シードを42に固定する
set_seed(42)

# Trainerに渡す引数を初期化する
training_args = TrainingArguments(
    output_dir="output_bert_ner", # 結果の保存フォルダ
    per_device_train_batch_size=32, # 訓練時のバッチサイズ
    per_device_eval_batch_size=32, # 評価時のバッチサイズ
    learning_rate=1e-4, # 学習率
    lr_scheduler_type="linear", # 学習率スケジューラ
    warmup_ratio=0.1, # 学習率のウォームアップ
    num_train_epochs=5, # 訓練エポック数
    evaluation_strategy="epoch", # 評価タイミング
    save_strategy="epoch", # チェックポイントの保存タイミング
    logging_strategy="epoch", # ロギングのタイミング
    fp16=False, # 自動混合精度演算の有効化
    no_cuda=True,
)

# Trainerを初期化する
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    args=training_args,
)

# 訓練する
trainer.train()

/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/transformers/training_args.py:1540: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


  0%|          | 0/670 [00:00<?, ?it/s]